![enter image description here](https://img.shields.io/badge/Author-Nestor_Batista_Díaz-yellow)

# Laboratorio: Modelos del lenguaje con RNNs

En este laboratorio, vamos a entrenar un modelo del lenguaje basado en caracteres con Recurrent Neural Networks. Asimismo, utilizaremos el modelo para generar texto. En particular, alimentaremos nuestro modelo con obras de la literatura clásica en castellano para obtener una red neuronal que sea capaz de "escribir" fragmentos literarios.

Los entrenamientos en esta laboratorio para obtener un modelo de calidad podrían tomar cierto tiempo (5-10 minutos por epoch), por lo que se aconseja empezar a trabajar pronto. El uso de GPUs no ayuda tanto con LSTMs como con CNNs, por lo que si tenéis máquinas potentes en casa es posible que podáis entrenar más rápido o a la misma velocidad que en Colab. En todo caso, la potencia de Colab es más que suficiente para completar este laboratorio con éxito.

<center><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/c/cb/Portada_Trafalgar_%281873%29.jpg/800px-Portada_Trafalgar_%281873%29.jpg" style="text-align: center" height="300px"></center>

El dataset a utilizar consistirá en un archivo de texto con el contenido íntegro en castellano de Trafalgar, disponible de manera libre en la página de [Project Gutenberg](https://www.gutenberg.org). Asimismo, como apartado optativo en este laboratorio se pueden utilizar otras fuentes de texto. Aquí podéis descargar los datos a utilizar de El Quijote y un par de obras adicionales:

[El ingenioso hidalgo Don Quijote de la Mancha (Miguel de Cervantes)](https://onedrive.live.com/download?cid=C506CF0A4F373B0F&resid=C506CF0A4F373B0F%219424&authkey=AH0gb-qSo5Xd7Io)

[Compilación de obras teatrales (Calderón de la Barca)](https://onedrive.live.com/download?cid=C506CF0A4F373B0F&resid=C506CF0A4F373B0F%219433&authkey=AKvGD6DC3IRBqmc)

[Trafalgar (Benito Pérez Galdós)](https://onedrive.live.com/download?cid=C506CF0A4F373B0F&resid=C506CF0A4F373B0F%219434&authkey=AErPCAtMKOI5tYQ)

Como ya deberíamos de estar acostumbrados en problemas de Machine Learning, es importante echar un vistazo a los datos antes de empezar.

## 1. Carga y procesado del texto

Primero, vamos a descargar el libro e inspeccionar los datos. El fichero a descargar es una versión en .txt del libro Trafalgar, a la cual se le han borrado introducciones, licencias y otras secciones para dejarlo con el contenido real de la novela.

In [134]:
import numpy as np
import keras
import matplotlib.pyplot as plt
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
import random
import io
import nltk
path = keras.utils.get_file(
    fname="Trafalgar.txt",
        origin="https://onedrive.live.com/download?cid=C506CF0A4F373B0F&resid=C506CF0A4F373B0F%219434&authkey=AErPCAtMKOI5tYQ",
        )


Una vez descargado, vamos a leer el contenido del fichero en una variable. Adicionalmente, convertiremos el contenido del texto a minúsculas para ponérselo un poco más fácil a nuestro modelo (de modo que todas las letras sean minúsculas y el modelo no necesite diferenciar entre minúsculas y mayúsculas).

**1.1.** Leer todo el contenido del fichero en una única variable ***text*** y convertir el string a minúsculas

In [135]:
text=open(path, encoding="utf8").read().lower()


Podemos comprobar ahora que efectivamente nuestra variable contiene el resultado deseado, con el comienzo tan característico del Quijote.

In [136]:
print("Longitud del texto: {}".format(len(text)))
print(text[0:300])


Longitud del texto: 300039
-i-

se me permitirá que antes de referir el gran suceso de que fui testigo,
diga algunas palabras sobre mi infancia, explicando por qué extraña
manera me llevaron los azares de la vida a presenciar la terrible
catástrofe de nuestra marina.

al hablar de mi nacimiento, no imitaré a la mayor parte de


## 2. Procesado de los datos

Una de las grandes ventajas de trabajar con modelos que utilizan caracteres en vez de palabras es que no necesitamos tokenizar el texto (partirlo palabra a palabra). Nuestro modelo funcionará directamente con los caracteres en el texto, incluyendo espacios, saltos de línea, etc.

Antes de hacer nada, necesitamos procesar el texto en entradas y salidas compatibles con nuestro modelo. Como sabemos, un modelo del lenguaje con RNNs acepta una serie de caracteres y predice el siguiente carácter en la secuencia.

* "*Se me permitirá que antes de referir el gr*" -> predicción: **a**
* "*e me permitirá que antes de referir el gra*" -> predicción: **n**

De modo que la entrada y la salida de nuestro modelo necesita ser algo parecido a este esquema. En este punto, podríamos usar dos formas de preparar los datos para nuestro modelo.

1. **Secuencia a secuencia**. La entrada de nuestro modelo sería una secuencia y la salida sería esa secuencia trasladada un caracter a la derecha, de modo que en cada instante de tiempo la RNN tiene que predecir el carácter siguiente. Por ejemplo:

>* *Input*:  Se me permitirá que antes de referir el gr
>* *Output*: e me permitirá que antes de referir el gra

2. **Secuencia a carácter**. En este variante, pasaríamos una secuencia de caracteres por nuestra RNN y, al llegar al final de la secuencia, predeciríamos el siguiente carácter.

>* *Input*:  Se me permitirá que antes de referir el gr
>* *Output*: a

En este laboratorio, por simplicidad, vamos a utilizar la segunda variante (PERO USANDO PALABRAS NO CARACTERES).

De este modo, a partir del texto, hemos de generar nuestro propio training data que consista en secuencias de PALABRAS con la siguiente PALABRA a predecir. Para estandarizar las cosas, utilizaremos secuencias de tamaño *SEQ_LENGTH* PALABRAS (un hiperparámetro que podemos elegir nosotros).



#### 2.1. Obtención de las palabras y mapas de palabras

Antes que nada, necesitamos saber qué palabras aparecen en el texto, ya que tendremos que diferenciarlos mediante un índice de 0 a *num_words* - 1 en el modelo. Obtener:


1.   Número de caracteres únicos que aparecen en el texto.
2.   Diccionario que asocia char a índice único entre 0 y *num_words* - 1. Por ejemplo, {'se': 0, 'me': 1, ...}
3.   Diccionario reverso de índices a palabras: {0: 'se', 1: 'me', ...}


In [137]:
texto=""
for letra in text:
    if not letra in "=><*_?¿.,'¡!()-{};[]»«:\"":
        texto+=letra
    text=texto

palabras = text.split()
palabras_unicas = set(palabras)
word_index = {}
reversed_index = {}
k=0
for palabra in palabras_unicas:
    word_index[palabra] = k
    reversed_index[k] = palabra
    k+=1

In [138]:
len(palabras_unicas)

8688

In [139]:
word_index

{'fruta': 0,
 'significar': 1,
 'coleto': 2,
 'apocamiento': 3,
 'hice': 4,
 'entibiaba': 5,
 'sargento': 6,
 'aparente': 7,
 'encopetados': 8,
 'busqué': 9,
 'mudo': 10,
 'datos': 11,
 'cuándo': 12,
 'entrometa': 13,
 'contestó': 14,
 'compramos': 15,
 'desencadenó': 16,
 'recrudecimiento': 17,
 'conociéramos': 18,
 'carena': 19,
 'resultados': 20,
 'éstos': 21,
 'cordero': 22,
 'castaña': 23,
 'conocerá': 24,
 'reñido': 25,
 'apto': 26,
 'fiestas': 27,
 'largamente': 28,
 'marché': 29,
 'novena': 30,
 'baile': 31,
 'pretendía': 32,
 'serios': 33,
 'comparativamente': 34,
 'entraremos': 35,
 'sequedad': 36,
 'crueldades': 37,
 'hallándome': 38,
 'desahogo': 39,
 'ochenta': 40,
 'agilidad': 41,
 'civil': 42,
 'cerró': 43,
 'marchar': 44,
 'maniobrar': 45,
 'amada': 46,
 'argumosa': 47,
 'malestar': 48,
 'edad': 49,
 'scipion': 50,
 'apreciarlo': 51,
 'zapatos': 52,
 'arreglaban': 53,
 'envolviéndonos': 54,
 'pecadillos': 55,
 'ahogados': 56,
 'estrechaba': 57,
 'apresados': 58,
 'despe

#### 2.2. Obtención de secuencias de entrada y palabra a predecir

Ahora, vamos a obtener las secuencias de entrada en formato texto y las correspondientes palabras a predecir. Para ello, recorrer el texto completo leído anteriormente, obteniendo una secuencia de SEQ_LENGTH palabras y la siguiente palabra a predecir. Una vez hecho, desplazarse una palabra a la derecha y hacer lo mismo para obtener una nueva secuencia y predicción. Guardar las secuencias en una variable ***sequences*** y las palabras a predecir en una variable ***next_words***.

Por ejemplo, si el texto fuera "Don Quijote" y SEQ_LENGTH fuese 2, tendríamos

* *sequences* = ["Don Quijote", "Quijote de"]
* *next_chars* = ['de', 'La']

In [140]:
# Definimos el tamaño de las secuencias. Puedes dejar este valor por defecto.
SEQ_LENGTH = 5
NUM_WORDS = len(palabras_unicas)

step=1
sequences = []
next_words = []

Indicar el tamaño del training set que acabamos de generar.

In [141]:
for i in range(0,len(palabras)-SEQ_LENGTH, step):
    sequences.append(palabras[i:i+SEQ_LENGTH])
    next_words.append(palabras[i+SEQ_LENGTH])


In [142]:
sequences[8]


['el', 'gran', 'suceso', 'de', 'que']

In [143]:
next_words[8]

'fui'

Como el Quijote es muy largo y tenemos muchas secuencias, podríamos encontrar problemas de memoria. Por ello, vamos a elegir un número máximo de ellas. Si estás corriendo esto localmente y tienes problemas de memoria, puedes reducir el tamaño aún más, pero ten cuidado porque, a menos datos, peor calidad del modelo.

In [144]:
sequences[len(sequences)-1]

['ilustrada', 'por', 'enrique', 'y', 'arturo']

In [145]:
next_words[len(sequences)-1]


'mélida'

In [146]:
#Convertimos las sequencias en frases
frases = []
for i in range(len(sequences)):
    frase = ' '.join(sequences[i])
    frases.append(frase)
#Tokenizar las frases
newsVec = [nltk.word_tokenize(frase, language= "spanish") for frase in frases]


In [147]:
newsVec

[['i', 'se', 'me', 'permitirá', 'que'],
 ['se', 'me', 'permitirá', 'que', 'antes'],
 ['me', 'permitirá', 'que', 'antes', 'de'],
 ['permitirá', 'que', 'antes', 'de', 'referir'],
 ['que', 'antes', 'de', 'referir', 'el'],
 ['antes', 'de', 'referir', 'el', 'gran'],
 ['de', 'referir', 'el', 'gran', 'suceso'],
 ['referir', 'el', 'gran', 'suceso', 'de'],
 ['el', 'gran', 'suceso', 'de', 'que'],
 ['gran', 'suceso', 'de', 'que', 'fui'],
 ['suceso', 'de', 'que', 'fui', 'testigo'],
 ['de', 'que', 'fui', 'testigo', 'diga'],
 ['que', 'fui', 'testigo', 'diga', 'algunas'],
 ['fui', 'testigo', 'diga', 'algunas', 'palabras'],
 ['testigo', 'diga', 'algunas', 'palabras', 'sobre'],
 ['diga', 'algunas', 'palabras', 'sobre', 'mi'],
 ['algunas', 'palabras', 'sobre', 'mi', 'infancia'],
 ['palabras', 'sobre', 'mi', 'infancia', 'explicando'],
 ['sobre', 'mi', 'infancia', 'explicando', 'por'],
 ['mi', 'infancia', 'explicando', 'por', 'qué'],
 ['infancia', 'explicando', 'por', 'qué', 'extraña'],
 ['explicando', 'p

In [148]:
from gensim.models import Word2Vec
model=Word2Vec(newsVec,min_count=1,vector_size=32)

In [149]:
model.wv["testigo"]


array([ 2.7635135e-02, -2.7069360e-01,  2.5417536e-01,  5.3740088e-02,
       -2.1682829e-02, -2.1519803e-01,  1.3474883e-01,  3.7878979e-02,
       -1.7135926e-01,  9.7342245e-02,  7.6183595e-02, -1.0444162e-01,
        1.2932117e-01, -2.0334028e-01, -1.9326811e-01,  7.4618519e-03,
        9.2466949e-03,  8.1479929e-02, -6.2993072e-02,  4.6322001e-03,
        2.8416273e-01,  2.6870310e-01,  2.6627645e-01,  4.8536900e-02,
       -8.4870614e-02,  2.2298826e-02, -1.9442376e-01,  1.8887356e-01,
       -2.7533177e-01, -2.5968352e-01, -2.1515517e-01, -4.2733500e-05],
      dtype=float32)

In [150]:
model.wv.most_similar("testigo")


[('hablar', 0.9915086627006531),
 ('hoy', 0.9911240935325623),
 ('estas', 0.9911211133003235),
 ('ellos', 0.9909572005271912),
 ('aquellas', 0.9908859729766846),
 ('estar', 0.9908157587051392),
 ('mortal', 0.9907791614532471),
 ('mezcladas', 0.9906349778175354),
 ('encontré', 0.9904002547264099),
 ('terror', 0.9903675317764282)]

#### 2.3. Obtención de input X y output y para el modelo

Finalmente, a partir de los datos de entrenamiento que hemos generado vamos a crear los arrays de datos X e y que pasaremos a nuestro modelo.

Para ello, vamos a utilizar *one-hot encoding* para nuestras palabras. Por ejemplo, si sólo tuviéramos 4 palabras (a, b, c, d), las representaciones serían: (1, 0, 0, 0), (0, 1, 0, 0), (0, 0, 1, 0) y (0, 0, 0, 1).

De este modo, **X** tendrá shape *(num_sequences, seq_length, num_words)* e **y** tendrá shape *(num_sequences, num_words)*.



In [151]:
NUM_SEQUENCES = len(sequences)
X = np.zeros((NUM_SEQUENCES, SEQ_LENGTH, 32))
y = np.zeros((NUM_SEQUENCES, NUM_WORDS))


In [152]:
for k in range(NUM_SEQUENCES):
    palabra=next_words[k]
    y[k,word_index[palabra]]=1 #codificación ONE-HOT

In [153]:
for k in range(NUM_SEQUENCES):
  for w in range(SEQ_LENGTH):
    word=sequences[k][w]
    X[k,w]=model.wv[word]



## 3. Definición del modelo y entrenamiento

Una vez tenemos ya todo preparado, es hora de definir el modelo. Define un modelo que utilice una **LSTM** con **128 unidades internas**. Si bien el modelo puede definirse de una manera más compleja, para empezar debería bastar con una LSTM más una capa Dense con el *softmax* que predice el siguiente caracter a producir. Adam puede ser una buena elección de optimizador.

Una vez el modelo esté definido, entrénalo un poco para asegurarte de que la loss es decreciente. No es necesario guardar la salida de este entrenamiento en el entregable final, ya que vamos a hacer el entrenamiento más informativo en el siguiente punto.

In [154]:
vocabulary_size=len(palabras_unicas)
model=Sequential()
model.add(LSTM(128, input_shape=(SEQ_LENGTH,32)))
model.add(Dense(vocabulary_size,activation='softmax'))
print(model.summary())


C:\Users\maqui\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_7 (LSTM)                   │ (None, 128)            │        82,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 8688)           │     1,120,752 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,203,184 (4.59 MB)

 Trainable params: 1,203,184 (4.59 MB)

 Non-trainable params: 0 (0.00 B)

None


In [155]:
optimizer='adam'
model.compile(loss='categorical_crossentropy',optimizer=optimizer,metrics=['accuracy'])
history=model.fit(X,y, validation_split=0.05, batch_size=128, epochs=50,shuffle=True)

Epoch 1/50
381/381 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - accuracy: 0.0421 - loss: 7.4245 - val_accuracy: 0.0715 - val_loss: 7.2944
Epoch 2/50
381/381 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.0663 - loss: 6.6868 - val_accuracy: 0.0738 - val_loss: 7.3292
Epoch 3/50
381/381 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.0710 - loss: 6.5020 - val_accuracy: 0.0754 - val_loss: 7.2637
Epoch 4/50
381/381 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.0770 - loss: 6.3218 - val_accuracy: 0.0836 - val_loss: 7.1952
Epoch 5/50
381/381 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.0838 - loss: 6.1384 - val_accuracy: 0.0836 - val_loss: 7.2278
Epoch 6/50
381/381 ━━━━━━━━━━━━━━━━━━━━ 6s 14ms/step - accuracy: 0.0851 - loss: 5.9848 - val_accuracy: 0.0886 - val_loss: 7.2380
Epoch 7/50
381/381 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.0891 - loss: 5.8428 - val_accuracy: 0.0886 - val_loss: 7.2424
Epoch 8/50
381/381 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.0903 - loss: 5.7083 - val_accu

In [156]:
X[3]


array([[ 0.01930262, -0.19566682,  0.2257279 ,  0.06303161,  0.01262652,
        -0.18627235,  0.06547091,  0.05046972, -0.20360892,  0.09259841,
         0.05743712, -0.07132103,  0.12050421, -0.17962825, -0.15199918,
         0.01269041,  0.04923926,  0.09117057, -0.07236584, -0.0219085 ,
         0.19655742,  0.22319795,  0.20072809,  0.04690018, -0.02702017,
         0.0613173 , -0.11653874,  0.18541923, -0.2169382 , -0.19354765,
        -0.17033638, -0.04739213],
       [ 0.21490508, -0.76450717,  0.93610668,  0.29672828,  0.17693943,
        -0.75144857, -0.19112235, -0.31506678, -1.58886302,  0.12955248,
        -0.19639248, -0.65210128,  0.52199274, -0.63139498, -0.94340909,
         0.42641574,  0.17818014,  0.95150191, -0.81872308, -0.32438079,
         1.12329292,  0.84291506,  1.10929203, -0.21628238, -0.49928141,
        -0.13675663, -0.352052  ,  1.56973064, -1.71231389, -0.1133303 ,
        -0.89783883, -0.49375492],
       [-0.03791895, -1.10079336,  1.13470316,  0.1265

Para ver cómo evoluciona nuestro modelo del lenguaje, vamos a generar texto según va entrenando. Para ello, vamos a programar una función que, utilizando el modelo en su estado actual, genere texto, con la idea de ver cómo se va generando texto al entrenar cada epoch.

En el código de abajo podemos ver una función auxiliar para obtener valores de una distribución multinomial. Esta función se usará para muestrear el siguiente carácter a utilizar según las probabilidades de la salida de softmax (en vez de tomar directamente el valor con la máxima probabilidad, obtenemos un valor aleatorio según la distribución de probabilidad dada por softmax, de modo que nuestros resultados serán más diversos, pero seguirán teniendo "sentido" ya que el modelo tenderá a seleccionar valores con más probabilidad).



In [157]:
def sample(probs, temperature=1.0):
    """Nos da el índice del elemento a elegir según la distribución
    de probabilidad dada por probs.
    Args:
    probs es la salida dada por una capa softmax:
    probs = model.predict(x_to_predict)[0]
    temperature es un parámetro que nos permite obtener mayor
    "diversidad" a la hora de obtener resultados.
    temperature = 1 nos da la distribución normal de softmax
    0 < temperature < 1 hace que el sampling sea más conservador,
    de modo que sampleamos cosas de las que estamos más seguros
    temperature > 1 hace que los samplings sean más atrevidos,
    eligiendo en más ocasiones clases con baja probabilidad.
    Con esto, tenemos mayor diversidad pero se cometen más
    errores.
    """
    # Cast a float64 por motivos numéricos
    probs = np.asarray(probs).astype('float64')
    # Hacemos logaritmo de probabilidades y aplicamos reducción
    # por temperatura.
    probs = np.log(probs) / temperature
    # Volvemos a aplicar exponencial y normalizamos de nuevo
    exp_probs = np.exp(probs)
    probs = exp_probs / np.sum(exp_probs)
    # Hacemos el sampling dadas las nuevas probabilidades
    # de salida (ver doc. de np.random.multinomial)
    samples = np.random.multinomial(1, probs, 1)
    return np.argmax(samples)

Utilizando la función anterior y el modelo entrenado, vamos a añadir un callback a nuestro modelo para que, según vaya entrenando, veamos los valores que resultan de generar textos con distintas temperaturas al acabar cada epoch.

Para ello, abajo tenéis disponible el callback *on_epoch_end*. Esta función elige una secuencia de texto al azar en el texto disponible en la variable
text y genera textos de longitud *GENERATED_TEXT_LENGTH* según las temperaturas en *TEMPERATURES_TO_TRY*, utilizando para ello la función *generate_text*.

Completa la función *generate_text* de modo que utilicemos el modelo y la función sample para generar texto.

NOTA: Cuando hagas model.predict, es aconsejable usar verbose=0 como argumento para evitar que la función imprima valores de salida.

In [158]:
# TEMPERATURES_TO_TRY = [0.2, 0.5, 1.0, 1.2]
TEMPERATURES_TO_TRY = [0.2]
GENERATED_TEXT_LENGTH = 10


In [159]:
def generate_text(seed_text, model, length, temperature=1):
    """Genera una secuencia de texto a partir de seed_text utilizando model.
    La secuencia tiene longitud length y el sampling se hace con la temperature
    definida.
    """
    # Aquí guardaremos nuestro texto generado, que incluirá el
    # texto origen
    generated = seed_text
    # Utilizar el modelo en un bucle de manera que generemos
    # carácter a carácter. Habrá que construir los valores de
    # X_pred de manera similar a como hemos hecho arriba, salvo que
    # aquí sólo se necesita una oración
    # Nótese que el x que utilicemos tiene que irse actualizando con
    # los caracteres que se van generando. La secuencia de entrada al
    # modelo tiene que ser una secuencia de tamaño SEQ_LENGTH que
    # incluya el último caracter predicho.
    ### TU CÓDIGO AQUÍ

    # Dividimos las frases en palabras y las alamacenamos en una lista
    seed_words = seed_text.split()
    for z in range(length):
      # Contamos el número de palabras en la frase
      NUM_SEED_WORDS = len(seed_words)

      # Creamos la matriz X_pred con los tamaños("número de palabras", "tamaño de la secuencia (en este caso sera igual al número de palabras), 32 (Que es con el valor con el cual hemos entrenado el modelo)")
      X_pred = np.zeros((NUM_SEED_WORDS, SEQ_LENGTH, 32))

      # Volvemos a formar la frase con las palabras que se han ido generando
      frase = " ".join(str(word) for word in seed_words)

      #Tokenizar las frases
      news_seed_Vec = [nltk.word_tokenize(frase, language= "spanish")]
      
      # Entrenamos con Word2Vec
      gensim_model=Word2Vec(news_seed_Vec,min_count=1,vector_size=32)

      # Almacenamos en la matriz a predecir el codigo ONE-HOT
      for k in range(NUM_SEED_WORDS):
        for w in range(SEQ_LENGTH):
          word=seed_words[k]
          X_pred[k,w]=gensim_model.wv[word]
    
      # Predecimos
      prediccion=model.predict(X_pred, batch_size=32, verbose=0)

      # Actualizamos la lista de palabras de entrada
      seed_words.pop(0)
      # Añadimos la nueva palabra predicha a la lista
      seed_words.append(reversed_index[sample(prediccion[0],temperature)])
      # Añadimos la palabra predicha a la cadena que se mostrara al final
      generated+=" "+reversed_index[sample(prediccion[0],temperature)]
    ### FIN DE TU CÓDIGO
    return generated


In [160]:
def on_epoch_end(epoch, logs):
    print("\n\n\n")
    # Primero, seleccionamos una palabra al azar para empezar a predecir
    start_pos = random.randint(0, len(palabras) - SEQ_LENGTH)
    # Generamos la frase inicial a predecir  
    seed_text = " ".join(palabras[start_pos:start_pos + SEQ_LENGTH])
    for temperature in TEMPERATURES_TO_TRY:
        print("------> Epoch: {} - Generando texto con temperature {}".format(
        epoch + 1, temperature))
        generated_text = generate_text(seed_text, model,
        GENERATED_TEXT_LENGTH, temperature)
        print("Seed: {}".format(seed_text))
        print("Texto generado: {}".format(generated_text))
        print()

        print("\n\n\n")



In [161]:
generation_callback = LambdaCallback(on_epoch_end=on_epoch_end)

Entrena ahora tu modelo. No te olvides de añadir *generation_callback* a la lista de callbacks utilizados en fit(). Ya que las métricas de clasificación no son tan críticas aquí (no nos importa tanto acertar el carácter exacto, sino obtener una distribución de probabilidad adecuada), no es necesario monitorizar la accuracy ni usar validation data, si bien puedes añadirlos para asegurarte de que todo está en orden.


In [162]:
history=model.fit(X,y, validation_split=0.05, batch_size=128, epochs=10,shuffle=True,verbose=0,callbacks=[generation_callback])





------> Epoch: 1 - Generando texto con temperature 0.2
Seed: la tarde se me representan
Texto generado: la tarde se me representan resolución resolución resolución resolución resolución resolución resolución resolución resolución resolución









------> Epoch: 2 - Generando texto con temperature 0.2
Seed: los mayores del mundo eran
Texto generado: los mayores del mundo eran capa resolución forjado forjado forjado resolución resolución resolución resolución resolución









------> Epoch: 3 - Generando texto con temperature 0.2
Seed: como he dicho un cuarto
Texto generado: como he dicho un cuarto resolución resolución forjado forjado resolución resolución resolución resolución resolución resolución









------> Epoch: 4 - Generando texto con temperature 0.2
Seed: según lo que pedían las
Texto generado: según lo que pedían las resolución resolución forjado forjado forjado resolución dominaba resolución forjado forjado









------> Epoch: 5 - Generando texto con tempera

## Entregable

Completa los apartados anteriores para entrenar modelos del lenguaje que sean capaces de generar texto con cierto sentido. Comentar los resultados obtenidos y cómo el modelo va mejorando época a época. Comentar las diferencias apreciadas al utilizar diferentes valores de temperatura. Entregar al menos la salida de un entrenamiento completo con los textos generados época a época.

El objetivo no es conseguir generar pasajes literarios con coherencia, sino obtener lenguaje que se asemeje en cierta manera a lo visto en el texto original y donde las palabras sean reconocibles como construcciones en castellano. Como ejemplo de lo que se puede conseguir, este es el resultado de generar texto después de 10 epochs y con temperature 0.2 usando El Quijote:


```
-----> Epoch: 10 - Generando texto con temperature 0.2
Seed: o le cautivaron y rindieron el
Texto generado: o le cautivaron y rindieron el caballero de la caballería de la mano de la caballería del cual se le dijo:

-¿quién es el verdad de la caballería de la caballería de la caballería de la caballería de la caballería, y me ha de habían de la mano que el caballero de la mano de la caballería. y que no se le habían de la mano de la c

```

Asimismo, se proponen los siguientes aspectos opcionales para conseguir nota extra:

*   Experimentar con los textos de teatro en verso de Calderón de la Barca (¿es capaz el modelo de aprender las estructuras del teatro en verso?) o con alguno de los otros textos disponibles. También se puede probar con textos de vuestra elección.
*   Experimentar con distintos valores de SEQ_LENGTH.
*   Experimentar con los hiperparámetros del modelo o probar otro tipo de modelos como GRUs o *stacked* RNNs (RNNs apiladas).
*   Experimentar utilizando embeddings en vez de representaciones one-hot.
*   (Difícil) Entrenar un modelo secuencia a secuencia en vez de secuencia a carácter.


